In [1]:
#import all libraries
import pandas as pd
import requests
import bs4
import re
import zipfile
import logging
import concurrent.futures
import math
from time import sleep
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed

In [2]:
def get_links():
    urls = list(pd.read_csv('books_url.csv')['link'])
    return urls

In [3]:
def get_soup(input_url):
    headers = {
        'User-Agent': 'My User Agent 1.0',
        "Accept-Language": "en-US,en;q=0.5"
    }
    response = requests.get(input_url, headers=headers)
    if response.status_code != 200:
        print("Error in getting link")
        print("response code is : ", response.status_code)
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
    return soup

In [4]:
def get_fa_title(soup):
    title = soup.select('.product-name strong')[0]
    return title.text


In [5]:
def get_en_title(soup):
    title = soup.select('.product-name-englishname')[0]
    return title.text

In [6]:
def get_price(soup):
    price = soup.select('.col-md-7 .price-special')[0].text
    return int(price.replace(',', ''))  #TODO make the price integer

In [7]:
# def get_discount(soup):
#     try:
#         discount_price = int(soup.select('.col-md-12+ .clearfix .price-special')[0].text.replace(',', ''))
#         discount_price = ((get_price(soup) - discount_price) / get_price(soup)) * 100
#     except Exception:
#         discount_price = 0
#         logging.exception("This book has no discount!")
#     return discount_price  

############################################################################################
#plz check
############################################################################################

def get_discount(soup):
    cleaned_text=soup.select('li:nth-child(1) div')[0].text
    cleaned_text=re.sub('تخفیف','',cleaned_text)
    return cleaned_text

In [10]:
def get_score(soup):
    soup = soup.find('div', {'class':'col-md-7'}).find('li', {'class':'pull-left'}).find('div', {'class':'my-rating'})
    soup_str = str(soup)

    match = re.search(r'data-rating="(\d+\.\d+)"', soup_str)
    if match:
        data_rating = match.group(1)
        return data_rating
    

In [30]:
def get_publisher(soup):
    publisher_href = soup.find('div', {'class': 'row clearfix'}).find('a')['href']
    try:
        publisher = soup.select('.prodoct-attribute-items:nth-child(1) a .prodoct-attribute-item')[0].text.strip()
    except Exception:
        publisher = None
        logging.exception("This book has no publisher!")
    return publisher

In [ ]:
def get_author(soup):
    try:
        author = soup.select('.prodoct-attribute-items+ .prodoct-attribute-items a .prodoct-attribute-item')[0].text.strip()
    except Exception:
        author = None
        logging.exception("This book has no author!")    
    return author

In [ ]:
def get_code(soup):
    code = soup.select('tr:nth-child(1) td+ td')[0]
    return code.text

In [ ]:
def get_isbn(soup):
    isbn = soup.select('.product-table .ltr')[0]
    cleaned_isbn = isbn.text.strip()
    cleaned_isbn = re.sub('[^0-9-]', '', cleaned_isbn)
    return cleaned_isbn

In [ ]:
def get_pages(soup):
    pages = soup.select('tr:nth-child(5) .rtl')[0]
    cleaned_pages = pages.text.strip()
    cleaned_pages = re.sub('[^0-9-]', '', cleaned_pages)
    return cleaned_pages

In [ ]:
def get_per_date(soup):
    try:
        date = soup.select('.product-table > tbody:nth-child(1) > tr:nth-child(6) > td:nth-child(2)')[0].text
    except Exception:
        date = None
        logging.exception("This book has persian calendar!")
    return date.strip() 

In [ ]:
def get_ad_date(soup):
    pass #TODO
    return None

In [ ]:
def get_size(soup):
    size = soup.select(
        'html body div.container div div.product-container.well.clearfix div.clearfix div.col-md-9.col-sm-9 div.row div.col-md-5 table.product-table tbody tr td.rtl')[0].text
    return size.strip()


In [ ]:
def get_material(soup):
    pass #TODO
    return None


In [ ]:
def get_series(soup):
    pass #TODO
    return None


In [ ]:
def get_send_time(soup):
    pass
    return None


In [ ]:
def get_summary(soup):
    summary = soup.select('.product-description')[0].text.strip()
    return summary

In [ ]:
def get_tags(soup):
    tags = soup.select('.product-tags-item')
    tags_list = []
    for tag in tags:
        tags_list += [tag.text.strip()]
    return tags_list

In [ ]:
 def get_book_detail(book_soup, site_index):
    book_fa_title = get_fa_title(book_soup)
    book_en_title = get_en_title(book_soup)
    book_price = get_price(book_soup)
    book_discount_percent = get_discount(book_soup)
    book_score = get_score(book_soup)
    book_publisher = get_publisher(book_soup)
    book_author = get_author(book_soup)
    book_code = get_code(book_soup)
    book_Isbn = get_isbn(book_soup)
    book_pages = get_pages(book_soup)
    book_publication_per_date = get_per_date(book_soup)
    book_publication_ad_date = get_ad_date(book_soup)
    book_size = get_size(book_soup)
    book_cover_material = get_material(book_soup)
    book_print_series = get_series(book_soup)
    book_earliest_send_time = get_send_time(book_soup)

    book_data = [site_index, book_code, book_Isbn, book_fa_title, book_en_title, book_price, book_discount_percent,
                 book_score, book_publisher, book_author,
                 book_pages, book_publication_per_date, book_publication_ad_date, book_size, book_cover_material,
                 book_print_series, book_earliest_send_time]  #TODO
    return book_data

In [ ]:
 def get_book_site_summary(book_soup, site_index):
    try:
        book_summary = get_summary(book_soup)
    except Exception:
        book_summary = None
        logging.exception("This book has no summary!")
    return [site_index, book_summary]

In [ ]:
 def get_book_site_tags(book_soup, site_index):
    book_tags = get_tags(book_soup)
    book_tags_list = []
    for tag in book_tags:
        book_tags_list += [[site_index,tag]]
    return book_tags_list

In [ ]:
books_data_list = []
site_index = 1
site_summary_data_list = []
site_tags_data_list = []
links = get_links()[:10] + ['https://www.iranketab.ir/book/270-gone-with-the-wind']

for link in links:
    try:
        site_soup = get_soup(link)
        site_summary_data_list += [get_book_site_summary(site_soup, site_index)]
        for site_tag in get_book_site_tags(site_soup, site_index):
            site_tags_data_list += [site_tag]
        site_page_books = site_soup.select('.clearfix .clearfix .row')
        for book_index in range(0, len(site_page_books), 2):
            data = get_book_detail(site_page_books[book_index], site_index)
            books_data_list += [data];
        site_index += 1
    except Exception:
        logging.exception("An error occurred");

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import threading

books_data_list = []
site_summary_data_list = []
site_tags_data_list = []
links = get_links()[:100] + ['https://www.iranketab.ir/book/270-gone-with-the-wind']
site_index = 1
# Create a lock for the shared resource
lock = threading.Lock()

def process_link(link):
    try:
        site_soup = get_soup(link)
        with lock:
            global site_index
            site_summary_data_list.append(get_book_site_summary(site_soup, site_index))
            site_tags_data_list.extend(get_book_site_tags(site_soup, site_index))
            site_page_books = site_soup.select('.clearfix .clearfix .row')
            for book_index in range(0, len(site_page_books), 2):
                data = get_book_detail(site_page_books[book_index], site_index)
                books_data_list.append(data)
            site_index += 1
    except Exception:
        logging.exception("An error occurred")

# Use ThreadPoolExecutor to run the loop in parallel
with ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(process_link, links)


In [ ]:
tableOfData = pd.DataFrame(books_data_list,
                           columns=['site_index', 'code', 'Isbn', 'fa_title', 'en_title', 'price', 'discount', 'score',
                                    'publisher', 'author', 'pages', 'publication_per_date', 'publication_ad_date',
                                    'size', 'cover_material', 'print_series', 'earliest_send_time'])
tableOfData

In [ ]:
tableOfSummaryData = pd.DataFrame(site_summary_data_list, columns=['site_index', 'summary'])
tableOfSummaryData

In [ ]:
tableOfSiteTagsData = pd.DataFrame(site_tags_data_list, columns=['site_index', 'tag'])
tableOfSiteTagsData

In [ ]:
file_path = "data.csv"

tableOfData.to_csv(file_path, index=False, encoding='utf-8')